<a href="https://colab.research.google.com/github/Denis-Fa/JavaRushTasks/blob/master/Surprise_Shir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise
!pip install flask

Шаг 1: Импорт необходимых библиотек.
Для начала импортируем необходимые библиотеки, включая Surprise для создания и обучения модели рекомендательной системы.

In [ ]:
import random
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

from collections import defaultdict
from surprise import Dataset, SVD

from surprise.model_selection import GridSearchCV

from flask import Flask, jsonify


Шаг 2: Создание собственного датасета.
Cоздадим собственный датасет, который будет содержать информацию о пользователях, товарах и их оценках. В этом примере мы создадим случайные данные для пользователей и товаров.

In [ ]:
# Создаем список пользователей и инструментов
users = [str(i) for i in range(1, 101)]
tools = [str(i) for i in range(1, 201)]

# Генерируем случайные оценки пользователей для инструментов
data = []
for user in users:
    for tool in tools:
        rating = random.randint(1, 5)
        data.append([user, tool, rating])

# Создаем DataFrame из данных
df = pd.DataFrame(data, columns=['user_id', 'tool_id', 'rating'])

# Создаем объект Reader для определения формата данных
reader = Reader(rating_scale=(1, 5))

# Создаем датасет из DataFrame и объекта Reader
dataset = Dataset.load_from_df(df[['user_id', 'tool_id', 'rating']], reader)

Шаг 3: Разделение данных на обучающий и тестовый наборы.
Для оценки производительности модели разделим данные на обучающий и тестовый наборы.

In [ ]:
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)


Шаг 4: Создание и обучение модели.
Теперь создадим модель рекомендательной системы. В этом примере мы используем модель SVD.

In [ ]:
model = SVD()
model.fit(trainset)

Шаг 5: Получение прогнозов и оценка производительности модели.
Мы получим прогнозы на тестовом наборе и оценим производительность модели.

In [ ]:
predictions = model.test(testset)

mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

MAE:  1.2746
RMSE: 1.4929
MAE: 1.2746096661985558
RMSE: 1.492867037844062


Шаг 6: Получение рекомендаций для конкретного пользователя.
Теперь мы можем получить персонализированные рекомендации для конкретного пользователя. Для этого выберем случайного пользователя и получим рекомендации.

In [ ]:
# Выбираем случайного пользователя
user_id = random.choice(users)

from collections import defaultdict

from surprise import Dataset, SVD


def get_top_n(predictions, n=10):
    """Возвращает первые N рекомендаций для каждого пользователя из набора прогнозов.

     Аргументы:
         предсказания (список объектов предсказаний): список предсказаний, как
             возвращается тестовым методом алгоритма.
         n(int): количество рекомендаций для вывода для каждого пользователя. По умолчанию
             это 10.

     Возврат:
     Словарь, в котором ключи представляют собой идентификаторы пользователей (необработанные), а значения представляют собой списки кортежей:
         [(raw item id, rating estimation), ...] of size n.
    """

    # Сначала сопоставим прогнозы с каждым пользователем..
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Затем отсортируем прогнозы для каждого пользователя и извлечем k самых высоких.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Получаем топ N рекомендаций для пользователя
top_n = get_top_n(predictions, n=10)

print(f"Рекомендации инструментов для пользователя {user_id}:")
for tool_id, rating in top_n[user_id]:
    print(f"Инструмент ID: {tool_id}, Рейтинг: {rating}")


Рекомендации инструментов для пользователя 24:
Инструмент ID: 114, Рейтинг: 3.8615747011070995
Инструмент ID: 129, Рейтинг: 3.8567198404068463
Инструмент ID: 136, Рейтинг: 3.6891530725777666
Инструмент ID: 200, Рейтинг: 3.637350340731583
Инструмент ID: 22, Рейтинг: 3.6050154724394767
Инструмент ID: 43, Рейтинг: 3.5759054458954056
Инструмент ID: 29, Рейтинг: 3.4485852685387925
Инструмент ID: 74, Рейтинг: 3.442959889731693
Инструмент ID: 196, Рейтинг: 3.4087214686852114
Инструмент ID: 150, Рейтинг: 3.2843564721694256


Шаг 7: Настройка параметров модели.
Для настройки параметров модели, мы можем использовать методы подбора гиперпараметров, такие как GridSearchCV из scikit-learn.

In [ ]:
# Создаем объект модели SVD
model = SVD()

param_grid = {'n_factors': [50, 100, 150],
              'n_epochs': [20, 30, 40],
              'lr_all': [0.002, 0.005, 0.01],
              'reg_all': [0.02, 0.04, 0.06]}

# Используем GridSearchCV для подбора параметров
grid_search = GridSearchCV(SVD,
                           param_grid=param_grid,
                           measures=['rmse'],
                           cv=5)

# Выполняем GridSearchCV
grid_search.fit(dataset)

# Получаем наилучшие параметры
best_params = grid_search.best_params['rmse']
print(f'Наилучшие параметры: {best_params}')

# Создаем объект модели SVD с наилучшими параметрами
model = SVD(n_factors=best_params['n_factors'],
            n_epochs=best_params['n_epochs'],
            lr_all=best_params['lr_all'],
            reg_all=best_params['reg_all'])

# Обучаем модель с наилучшими параметрами
model.fit(trainset)

# Делаем предсказания на тестовом наборе
predictions = model.test(testset)

# Вычисляем показатели производительности
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')


Наилучшие параметры: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.06}
MAE:  1.2193
RMSE: 1.4149
MAE: 1.2193176047094967
RMSE: 1.41487189819824


Шаг 8: Интеграция в магазин инструментов.
Чтобы интегрировать рекомендательную систему в магазин, мы можем создать функцию, которая будет предоставлять рекомендации на основе предпочтений пользователя. Это может быть вызвано на странице пользователя или странице инструмента.

In [ ]:
def get_top_n(predictions, user_id, n=10):
    """Получение топ-N рекомендаций для пользователя"""
    top_n = defaultdict(list)
    for uid, iid, _, est, _ in predictions:
        if uid == user_id:
            top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n[user_id]

# Использование функции в магазине
user_id = '42'
recommended_toolss = get_top_n(predictions, user_id)


Для интеграции модели рекомендательной системы с веб-приложением на Java понадобится выполнить следующий шаг:

 Экспортирую обученную модель, созданную в коде Python, в файл (например, в формате pickle), чтобы ее можно было загрузить и использовать в Java. Для этого использую модуль pickle в Python:

In [ ]:
import pickle

# Сохранение обученной модели в файл

with open('model_surprise.pkl', 'wb') as file:
    pickle.dump(model, file)



 Создайте экземпляр объекта Flask:

In [47]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Welcome to machine learning model APIs!"

if __name__ == '__main__':
    app.run(debug=True)



 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
app = Flask(__name__)


3. Определите конечную точку вашего API (например, /predict) и определите функцию-обработчик для этой конечной точки. В этой функции вы будете использовать вашу обученную модель для делегирования запроса на предсказание:

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  # Принимаем данные в формате JSON
    user_id = data['user_id']

    # Используйте вашу обученную модель для выполнения предсказания
    # Получите рекомендации для заданного user_id
    # ...

    # Верните результат в формате JSON
    return jsonify(__name__)


4. Запустите приложение Flask:

In [ ]:
if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!curl ipecho.net/plain


34.73.151.156